In [41]:
import pandas as pd

In [42]:
data = pd.read_csv("E:\Product review Analysis\Reviews.csv")
data = data.sample(frac=1)[:20000]

In [43]:
data.columns = map(lambda x:x.lower(), list(data))
data["text"] = data["summary"] + " "+ data["text"]
data = data[["text", "score"]]

data.loc[data.score<3, "score"] = -1
data.loc[data.score==3, "score"] = 0
data.loc[data.score>3, "score"] = 1

data.head(5)


,text,score
109912,A Strong K-Cup of coffee Some people may feel ...,1
33228,Nice Tea I get a bag of the tea sent to me mon...,1
562251,delicious almonds! these almonds are totally a...,1
416646,Yummy I love flavored teas and am usually a li...,1
278106,Great jerky! This is an awesome product. The t...,1


In [44]:
data.tail(5)

,text,score
537423,Walnut and date These were the best nutrient b...,1
354759,perfect for halloween i ordered these to give ...,1
338782,Good Gravy! Very easy to make and tastes good ...,1
319076,"excellent coffee I love the coffee, light full...",1
511919,"Great quick, natural oatmeal at an amazing pri...",1


In [45]:
import random
import numpy
sentiment_data = zip(data["text"], data["score"])
s = list(sentiment_data)

In [46]:
random.shuffle(s)

# 80% for training
train_X, train_y = zip(*s[:16000])

# Keep 20% for testing
test_X, test_y = zip(*s[16000:])


In [47]:
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import pos_tag


In [48]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None



In [49]:
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import sent_tokenize, word_tokenize, pos_tag

lemmatizer = WordNetLemmatizer()

def sentiment_sentiwordnet(text):
    raw_sentences = sent_tokenize(text)
    sentiment = 0
    tokens_count = 0
    
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue

            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue

            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            word_sent = swn_synset.pos_score() - swn_synset.neg_score()

            if word_sent != 0:
                sentiment += word_sent
                tokens_count += 1

    if tokens_count == 0:
        return 0
    sentiment = sentiment/tokens_count
    if sentiment >= 0.01:
        return 1
    if sentiment <= -0.01:
        return -1
    return 0

In [50]:
from sklearn.metrics import accuracy_score

In [51]:
pred_y = [sentiment_sentiwordnet(text) for text in test_X]
accuracy_score(test_y, pred_y)

0.65425